# Section 4.2 — Multiple linear regression

This notebook contains the code examples from [Section 4.2 Multiple linear regression]() from the **No Bullshit Guide to Statistics**.

#### Notebook setup

In [1]:
# load Python modules
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Figures setup
plt.clf()  # needed otherwise `sns.set_theme` doesn't work
from plot_helpers import RCPARAMS
# RCPARAMS.update({'figure.figsize': (10, 3)})   # good for screen
RCPARAMS.update({'figure.figsize': (5, 1.6)})  # good for print
sns.set_theme(
    context="paper",
    style="whitegrid",
    palette="colorblind",
    rc=RCPARAMS,
)

# High-resolution please
%config InlineBackend.figure_format = 'retina'

# Where to store figures
DESTDIR = "figures/lm/multiple"

<Figure size 640x480 with 0 Axes>

In [3]:
# set random seed for repeatability
np.random.seed(42)

## Definitions

## Doctors dataset

In [4]:
doctors = pd.read_csv("../datasets/doctors.csv")
doctors

,permit,name,location,score
0,93636,Yesenia Smith,urban,82.0
1,79288,Andrew Stanley,rural,85.0
2,94980,Jessica Castro,rural,97.0
3,99441,Jasmine Lynch,urban,74.0
4,92882,Cassie Pena,urban,73.0
...,...,...,...,...
219,61425,Christian Garza,urban,80.0
220,70978,Rebecca Colon,urban,78.0
221,66559,Mario Salas,urban,78.0
222,33954,Aaron Jackson,rural,74.0
